In [ ]:
import deepxde as dde
import numpy as np
import matplotlib as plt
import torch
#dde.backend.set_default_backend('pytorch')

## PDE

Navier stokes 2d:

In [ ]:
rho = 1060  # density
eta = 0.005 # viscosity 

def pde_grad(x, y):
    u = y[:, 0:1] # x-coord
    v = y[:, 1:2] # y-coord
    p = y[:, 2:3] # pressure
    du_x = dde.grad.jacobian(y, x, i=0, j=0)
    du_y = dde.grad.jacobian(y, x, i=0, j=1)
    du_t = dde.grad.jacobian(y, x, i=0, j=2)
    dv_x = dde.grad.jacobian(y, x, i=1, j=0)
    dv_y = dde.grad.jacobian(y, x, i=1, j=1)
    dv_t = dde.grad.jacobian(y, x, i=1, j=2)
    dp_x = dde.grad.jacobian(y, x, i=2, j=0)
    dp_y = dde.grad.jacobian(y, x, i=2, j=1)
    du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    du_xy = dde.grad.hessian(y, x, component=0, i=0, j=1)
    du_xt = dde.grad.hessian(y, x, component=0, i=0, j=2)
    du_yx = dde.grad.hessian(y, x, component=0, i=1, j=0)
    du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    du_yt = dde.grad.hessian(y, x, component=0, i=1, j=2)
    du_tx = dde.grad.hessian(y, x, component=0, i=2, j=0)
    du_ty = dde.grad.hessian(y, x, component=0, i=2, j=1)
    du_tt = dde.grad.hessian(y, x, component=0, i=2, j=2)
    dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    dv_xy = dde.grad.hessian(y, x, component=1, i=0, j=1)
    dv_xt = dde.grad.hessian(y, x, component=1, i=0, j=2)
    dv_yx = dde.grad.hessian(y, x, component=1, i=1, j=0)
    dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)
    dv_yt = dde.grad.hessian(y, x, component=1, i=1, j=2)
    dv_tx = dde.grad.hessian(y, x, component=1, i=2, j=0)
    dv_ty = dde.grad.hessian(y, x, component=1, i=2, j=1)
    dv_tt = dde.grad.hessian(y, x, component=1, i=2, j=2)
    dp_xx = dde.grad.hessian(y, x, component=2, i=0, j=0)
    dp_xy = dde.grad.hessian(y, x, component=2, i=0, j=1)
    dp_xt = dde.grad.hessian(y, x, component=2, i=0, j=2)
    dp_yx = dde.grad.hessian(y, x, component=2, i=1, j=0)
    dp_yy = dde.grad.hessian(y, x, component=2, i=1, j=1)
    dp_yt = dde.grad.hessian(y, x, component=2, i=1, j=2)
    du_xxx = dde.grad.jacobian(du_xx, x, i=0, j=0)
    du_xxy = dde.grad.jacobian(du_xx, x, i=0, j=2)
    du_xxt = dde.grad.jacobian(du_xx, x, i=0, j=1)
    du_yyx = dde.grad.jacobian(du_yy, x, i=0, j=0)
    du_yyy = dde.grad.jacobian(du_yy, x, i=0, j=1)
    du_yyt = dde.grad.jacobian(du_yy, x, i=0, j=2)
    dv_xxx = dde.grad.jacobian(dv_xx, x, i=1, j=0)
    dv_xxy = dde.grad.jacobian(dv_xx, x, i=1, j=1)
    dv_xxt = dde.grad.jacobian(dv_xx, x, i=1, j=2)
    dv_yyx = dde.grad.jacobian(dv_yy, x, i=1, j=0)
    dv_yyy = dde.grad.jacobian(dv_yy, x, i=1, j=1)
    dv_yyt = dde.grad.jacobian(dv_yy, x, i=1, j=2)

    continuity = du_x + dv_y
    x_momentum = du_t + u*du_x + v*du_y + 1/rho*dp_x - eta*(du_xx + du_yy)
    y_momentum = dv_t + u*dv_x + v*dv_y + 1/rho*dp_y - eta*(dv_xx + dv_yy)

    cont_x = du_xx + dv_yx
    cont_y = du_xy + dv_yy
    cont_t = du_xt + du_yt
    x_mom_x = du_tx + u*du_xx + du_x*du_x + v*du_yx + dv_x*du_y + 1/rho*dp_xx - eta*(du_xxx + du_yyx)
    x_mom_y = du_ty + u*du_xy + du_y*du_x + v*du_yy + dv_y*du_y + 1/rho*dp_xy - eta*(du_xxy + du_yyy)
    x_mom_t = du_tt + u*du_xt + du_t*du_x + v*du_yt + dv_t*du_y + 1/rho*dp_xt - eta*(du_xxt + du_yyt)
    y_mom_x = dv_tx + u*dv_xx + du_x*dv_x + v*dv_yx + dv_x*du_y + 1/rho*dp_yx - eta*(dv_xxx + dv_yyx)
    y_mom_y = dv_ty + u*dv_xy + du_y*dv_x + v*dv_yy + dv_y*du_y + 1/rho*dp_yy - eta*(dv_xxy + dv_yyy)
    y_mom_t = dv_tt + u*dv_xt + du_t*dv_x + v*dv_yt + dv_t*dv_y + 1/rho*dp_yt - eta*(dv_xxt + dv_yyt)
    return [continuity, x_momentum, y_momentum, cont_x, cont_y, cont_t, x_mom_x, x_mom_y, x_mom_t, y_mom_x, y_mom_y, y_mom_t]


def pde(x,y):
    u = y[:, 0:1] # x-coord
    v = y[:, 1:2] # y-coord
    p = y[:, 2:3] # pressure
    du_x = dde.grad.jacobian(y, x, i=0, j=0)
    du_y = dde.grad.jacobian(y, x, i=0, j=1)
    du_t = dde.grad.jacobian(y, x, i=0, j=2)
    dv_x = dde.grad.jacobian(y, x, i=1, j=0)
    dv_y = dde.grad.jacobian(y, x, i=1, j=1)
    dv_t = dde.grad.jacobian(y, x, i=1, j=2)
    dp_x = dde.grad.jacobian(y, x, i=2, j=0)
    dp_y = dde.grad.jacobian(y, x, i=2, j=1)
    du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
    dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)
    continuity = du_x + dv_y
    x_momentum = du_t + u*du_x + v*du_y + 1/rho*dp_x - eta*(du_xx + du_yy)
    y_momentum = dv_t + u*dv_x + v*dv_y + 1/rho*dp_y - eta*(dv_xx + dv_yy)

    
    return [continuity, x_momentum, y_momentum]



## Domain

$[-0.03,0.03]\times[-0.12,0.12]\times[0,2]=\Omega$

or

$[0,0.06]\times[0,0.24]\times[0,2]=\Omega$

In [ ]:
x_min, x_max = 0, 0.06
y_min, y_max = 0, 0.24
t_min, t_max = 0, 2

# Spatial domain:
space_domain = dde.geometry.Rectangle([x_min, y_min], [x_max, y_max])
# Time domain: 
time_domain = dde.geometry.TimeDomain(t_min, t_max)
# Spatio-temporal domain
geomtime = dde.geometry.GeometryXTime(space_domain, time_domain)

## BC & IC

$u(0,y,t)=u(0.06,y) = 0$

$u(x,0,t)=277*x(0.06-x)$

In [ ]:
def parabolic(x):
    x_in = x[:,0:1]
    return 277.78 * x_in * (0.06 - x_in)
    #return 0.25

def inlet(x, on_boundary):
    return on_boundary and np.isclose(x[1], 0) # y = 0

def bot(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0) # x = 0

def top(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0.06) # x = 0.06


# BC for velocity in x-direction
u_bc1 = dde.icbc.DirichletBC(geomtime, lambda x: 0, top, component=0)
u_bc2 = dde.icbc.DirichletBC(geomtime, lambda x: 0, bot, component=0)
#u_inlet= dde.icbc.DirichletBC(geomtime, parabolic, inlet, component=0)

# BC for velocity in y-direction
v_bc1 = dde.icbc.DirichletBC(geomtime, lambda x: 0, top, component=1)
v_bc2 = dde.icbc.DirichletBC(geomtime, lambda x: 0, bot, component=1)
v_inlet= dde.icbc.DirichletBC(geomtime, parabolic, inlet, component=1)


## Data

In [ ]:
ratio = 0.869296
num_residuals = 1500

nd = int(num_residuals * ratio)
nb = int((num_residuals - nd) *7/8)
ni = int((num_residuals - nd) *1/8)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [v_inlet,
     u_bc1,
     u_bc2,
     v_bc1,
     v_bc2,
     ],
    num_domain=nd,
    num_boundary=nb,
    num_initial=ni,
)

## Create model (retrain as gPINN)

In [ ]:
layer_size = [3] + [329] * 2 + [3]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)
model = dde.Model(data, net)

## Training

In [ ]:
import time
lr = 0.001545
call = 0
model.compile("adam", lr=lr)
ta = time.time()
losshistory, train_state = model.train(iterations=10000)

#rar
for i in range(40):
    X = geomtime.random_points(1000)
    err_eq = np.abs(model.predict(X, operator=pde))[0]

    err = np.mean(err_eq)

    err_eq = torch.tensor(err_eq)
    x_ids = dde.backend.to_numpy(torch.topk(err_eq, 10, dim=0)[1])

    for elem in x_ids:
        print("Adding new point:", X[elem])
        data.add_anchors(X[elem])
    
    early_stopping = dde.callbacks.EarlyStopping(min_delta=1e-4, patience=2000)
    model.compile("adam", lr=1000)
    if i == 40 - 1:
        losshistory, _ = model.train(iterations=1000, disregard_previous_best=True, callbacks=[early_stopping], model_save_path="models/ns_it"+str(call)
        )
        call += 1
    else:
        losshistory, _ = model.train(iterations=1000, disregard_previous_best=True, callbacks=[early_stopping]
        )
texec = time.time() - ta


dde.saveplot(losshistory, train_state, issave=True, isplot=True)

## Plots

In [ ]:
import matplotlib.pyplot as plt
#points to test
xs = np.arange(0, 0.061, 0.001)
ys = np.arange(0, 0.241, 0.001)

#t_mid = int(t_max-t_min/2)
label = "velocity"
for t in [1]:
       data = np.array([np.array([x, y, t]) for x in xs for y in ys])
              
       pred = model.predict(data)
       data[:,2] = np.linalg.norm(pred[:,0:2],axis=1)

       fig = plt.figure()
       fig.set_figwidth(12)
       fig.set_figheight(12)
       ax1 = fig.add_subplot(projection='3d')
       ax1.set(title="t="+str(t),
              xlim=(x_min-0.001, x_max+0.001),  #note axis reversed
              ylim=(y_min-0.005, y_max+0.005),
              #zlim=(0,0.25),
              xlabel="x",
              ylabel="y",
              zlabel=label)
       #filt = p_start[p_start[:,2]>=0]
       #ax1.scatter(filt[:,0], filt[:,1], filt[:,2])
       ax1.scatter(data[:,0], data[:,1], data[:,2])
       plt.savefig("NS-"+label+"3d_t"+str(t*10)+".png",
              bbox_inches="tight",
              pad_inches=0.2,
              facecolor="white")
       plt.show()

In [ ]:
import matplotlib.pyplot as plt

xs = np.arange(0, 0.061, 0.001)
ys = np.arange(0, 0.241, 0.001)
t_mid = 0.2
data = np.array([np.array([x, y, t_mid]) for x in xs for y in ys])
       
pred = model.predict(data)
data[:,2] = np.linalg.norm(pred[:,0:2],axis=1)




fig1, ax2 = plt.subplots()
fig1.set_figwidth(16)
fig1.set_figheight(4)
CS = ax2.tricontourf(data[:,1], data[:,0], data[:,2], levels=1000, cmap="rainbow", #norm=normal
)
ax2.set(title="t="+str(t_mid),
       ylim=(x_min, x_max), 
       xlim=(y_min, y_max), 
)
ax2.set_xlabel("y", fontsize=16)
ax2.set_ylabel("x", fontsize=16)
ax2.tick_params(axis="both", labelsize=14)

fig1.colorbar(CS, label="velocity $[m/s]$")
plt.savefig("NS-velocity.png",
           bbox_inches="tight",
           pad_inches=0.2,
           facecolor="white")
plt.show()


## Plot

In [ ]:
import matplotlib.pyplot as plt
xs = np.arange(0,0.061,0.001)
ys = np.arange(0,0.25,0.01)

#t_mid = (t_max+t_min)/2
t_mid = 0.2
p_start = np.array([np.array([x, y, t_min]) for x in xs for y in ys]) # t=0
p_mid = np.array([np.array([x, y, t_mid]) for x in xs for y in ys]) # t=1
p_slut = np.array([np.array([x, y, t_max]) for x in xs for y in ys]) #t=2

pred = model.predict(p_mid) # functionsværdier i tiden t mid
p_mid[:,2] = pred[:,2]


fig1, ax2 = plt.subplots()
fig1.set_figwidth(16)
fig1.set_figheight(4)
CS = ax2.tricontourf(p_mid[:,1], p_mid[:,0], pred[:,2], levels=1000, cmap="rainbow")
ax2.set(title="t="+str(t_mid),
       ylim=(x_min, x_max), 
       xlim=(y_min, y_max), 
       xlabel="y",
       ylabel="x")
fig1.colorbar(CS, label="pressure $[N/m^2]$")



# ------------------------------------------------------------------------
xs = np.arange(0,0.07,0.005)
ys = np.arange(0,0.265,0.025)

#t_mid = (t_max+t_min)/2
t_mid = 0.2
p_start = np.array([np.array([x, y, t_min]) for x in xs for y in ys]) # t=0
p_mid = np.array([np.array([x, y, t_mid]) for x in xs for y in ys]) # t=1
p_slut = np.array([np.array([x, y, t_max]) for x in xs for y in ys]) #t=2

pred = model.predict(p_mid) # functionsværdier i tiden t=1

x = p_mid[:,0]
y = p_mid[:,1]
u = pred[:,0]
v = pred[:,1]
  
# Plotting Vector Field with QUIVER
ax2.quiver(y, x, v, u, color='g', width=0.003)
  
# Setting x, y boundary limits
plt.xlim(y_min, y_max)
plt.ylim(x_min, x_max)
  
# Show plot with grid
#plt.grid()
plt.savefig("NS-quiver2d.png",
           bbox_inches="tight",
           pad_inches=0.2,
           facecolor="white")
plt.show()